In [4]:
### import libraries
import requests
import os
from tqdm import tqdm
import multiprocessing as mp
from datetime import datetime, timedelta
from dotenv import load_dotenv

In [15]:
######## set the URL and credentials ########
# get the base path of the git repository
repo_dir = os.popen('git rev-parse --show-toplevel').read().strip()
###load the .env file
load_dotenv(dotenv_path=f"{repo_dir}/.env")

#get the environment vairables
base=os.getenv("BASE_FOLDER")

# URL for the whole dataset:
base_url = os.getenv("NEXTCLOUD_BASE")
# set the credentials
username = os.getenv("NEXTCLOUD_USERNAME")
password = os.getenv("NEXTCLOUD_PW")

print(f"Cloudbase: {base_url}")

######## set the storage path ########
# Create the directory if it does not exist
storage_path = repo_dir #base

Cloudbase: https://cloud.scadsai.uni-leipzig.de/index.php/s/gozxE5r9YdwGL8w/download


In [16]:
############# get the folder names #############
# Define the start and end dates
start_date = datetime(2020, 7, 4)
end_date = datetime(2020, 8, 4)

# Initialize vars
current_date = start_date
folder_names = []

# Loop through each day from the start date to the end date
while current_date <= end_date:
    date_str = current_date.strftime("%Y%m%d")
    date_str_2 = date_str+"_2"
    folder_names.append([date_str, date_str_2])

    # Move to the next day
    current_date += timedelta(days=1)

def flatten(xss):
    return [x for xs in xss for x in xs]
    
folder_names=flatten(folder_names)  

In [17]:
############# download the files #############
def request_folder(folder):
    
    #request the folder
    url= f"{base_url}?path=/&files={folder}"
    path = os.path.join(storage_path, f"{folder}.h5")
    
    if not os.path.exists(path):
        response = requests.get(url, auth=(username, password), stream=True)
    
        if response.status_code == 200:
            print("File downloading...")
            
            size = int(response.headers.get("Content-Length", 0))
            
            progress = tqdm(response.iter_content(1024), f"Downloading {path}", total=size, unit="B", unit_scale=True, unit_divisor=1024)
            
            with open(path, 'wb') as file:
                for data in progress.iterable:
                    file.write(data)
                    progress.update(len(data))      
                    
            print("File saved successfully at", path)  
        else:
            print(f"File {path} not found. Status code:", response.status_code)
    else:
        print(f"File {path} already exists.")

In [ ]:
##### multiprocessing #####
folder_names_sub = folder_names[:1]
pool=mp.Pool(mp.cpu_count()-4)
pool.map(request_folder, folder_names_sub)
pool.close()
pool.join()

In [19]:
request_folder(folder_names[:1][0])

File /home/sc.uni-leipzig.de/ju554xqou/big-data-praktikum/20200704.h5 already exists.


In [ ]:
##### remove all files #####
# for folder in folder_names:
#     path = os.path.join(storage_path, f"{folder}.h5")
#     if os.path.exists(path):
#         os.remove(path)
#         print(f"File {path} removed.")
#     else:
#         print(f"File {path} does not exist.")